In [0]:
import mlflow
import torch
import io
import numpy as np
from datasets import Dataset
from pyspark.sql.functions import col
from PIL import Image

def proprocess(example):
    """Convert binary image to a tensor of shape (3,224,224)"""
    image = Image.open(io.BytesIO(example['image'])).convert('RGB')
    
    ### Auto Image Processor ###
    # 1. resize
    image = image.resize((224,224)) #[224,224,3]
    
    # 2. Convert to Numpy array and normalize
    image_array = np.array(image, dtype=np.float32) / 255.0
    
    # 3. Transpose [224,224,3] -> [3,224,224]
    image_array = image_array.transpose(2,0,1)

    # Store as a PyTorch Tensor instead of a list
    example['pixel_values'] = torch.tensor(image_array, dtype=torch.float32)

    return example

dataset_name = 'training_dataset_augemented'
normal_samples = (
    spark.read.table(dataset_name)
    .filter(col('label')==1)
    .select('image', 'label')
    .limit(5)
)

# Spark DataFrame -> Hugging Face Dataset
normal_samples = Dataset.from_spark(normal_samples)
data = normal_samples.map(proprocess)

# Extract tensors and stack them
pixel_values_list = [torch.tensor(example['pixel_values']) for example in data]
input_tensor = torch.stack(pixel_values_list)

# print('First pixel_values_list :', pixel_values_list[0])
print('First pixel_values_list shape :', pixel_values_list[0].shape)
print(f'Final input tensor shape: {input_tensor.shape}') #(5,3,224,224)


# Load modle and move it to appropriate device(cpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_uri = 'runs:/feb6fb0cef6b40aeb2f4261329c8c3f9/model'
model = mlflow.pytorch.load_model(model_uri).to(device)
model.eval()

# Move input tensor to device and make prediction
input_tensor = input_tensor.to(device)

with torch.no_grad():
    predictions = model(input_tensor)

print(predictions)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

First pixel_values_list shape : torch.Size([3, 224, 224])
Final input tensor shape: torch.Size([5, 3, 224, 224])


2025-09-14 12:18:22.270592: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-14 12:18:22.459271: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-14 12:18:22.660419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-14 12:18:22.820214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-14 12:18:22.868759: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-14 12:18:23.178425: I tensorflow/core/platform/cpu_feature_gu

ImageClassifierOutput(loss=None, logits=tensor([[-1.0475,  0.7031],
        [ 0.4886,  2.9053],
        [-0.8897,  1.4195],
        [-1.6510,  1.9711],
        [-1.9251,  0.6765]]), hidden_states=None, attentions=None)


In [0]:
# Print output shape and predictions
print(f"output predictions: {predictions}")
print(f"Predictions.logits : {predictions.logits.shape} \n{predictions.logits}")

output predictions: ImageClassifierOutput(loss=None, logits=tensor([[-1.0475,  0.7031],
        [ 0.4886,  2.9053],
        [-0.8897,  1.4195],
        [-1.6510,  1.9711],
        [-1.9251,  0.6765]]), hidden_states=None, attentions=None)
Predictions.logits : torch.Size([5, 2]) 
tensor([[-1.0475,  0.7031],
        [ 0.4886,  2.9053],
        [-0.8897,  1.4195],
        [-1.6510,  1.9711],
        [-1.9251,  0.6765]])


In [0]:
import torch.nn.functional as F

# Class 0 -> "abnormal"
# Class 1 -> "normal"

# Apply softmax to get probabilities
probabilities = F.softmax(predictions.logits, dim=1)
print(f"Probabilities: {probabilities}")

Probabilities: tensor([[0.1480, 0.8520],
        [0.0819, 0.9181],
        [0.0904, 0.9096],
        [0.0260, 0.9740],
        [0.0690, 0.9310]])
